In [ ]:
#구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from math import radians, sin, cos, sqrt, atan2
import requests

from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.neighbors import NearestNeighbors
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

In [ ]:
# 공통 경로 설정
base_path = '/content/drive/MyDrive/Olist_ALL/'

# 각 데이터셋 로딩
cd = pd.read_csv(base_path + 'closed_cleaned.csv')
metrics_df = pd.read_csv(base_path + 'seller_metrics_all_fillna.csv')
merged_df = pd.read_csv(base_path + 'merged.csv')
categories_df = pd.read_csv(base_path + 'categories.csv')

In [ ]:
metrics_df

,seller_id,months_with_sale,recency,frequency,monetary,delayed_count,delay_rate,payment_mix_index,payment_concentration,unique_category_count,avg_review_score,total_reviews_x,within_2_days_count,avg_response_time,within_2_days_rate
0,0015a82c2db000af6aaaf3ae2ecb0532,2,315,3,2748.06,0,0.00,-0.00,1.00,1,3.67,3.0,2,1 days 11:50:14,0.67
1,001cca7ae9ae17fb1caed9dfb1094831,17,47,200,48349.22,12,0.06,0.94,0.77,2,3.90,236.0,171,1 days 21:22:06.593220338,0.72
2,002100f778ceb8431b7a1020ff7ab48f,8,139,51,2445.64,9,0.18,1.32,0.56,1,3.98,56.0,43,1 days 14:33:12,0.77
3,003554e2dce176b5555353e4f3555ac8,1,257,1,139.38,0,0.00,-0.00,1.00,0,5.00,1.0,1,0 days 09:05:47,1.00
4,004c9cd9d87a3c30c522c48c4fc07416,17,118,158,27810.35,10,0.06,0.88,0.79,1,4.13,173.0,127,2 days 03:16:29.653179190,0.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3048,ffc470761de7d0232558ba5e786e57b7,7,11,28,2519.44,1,0.04,1.20,0.63,8,4.30,30.0,24,1 days 14:37:10.500000,0.80
3049,ffdd9f82b9a447f6f8d4b91554cc7dd3,11,10,18,3607.52,0,0.00,1.19,0.68,2,4.25,20.0,12,1 days 19:04:00.250000,0.60
3050,ffeee66ac5d5a62fe688b9d26f83f534,7,107,14,2259.55,2,0.14,0.94,0.64,1,4.21,14.0,10,1 days 14:13:33.500000,0.71
3051,fffd5413c0700ac820c7069d66d98c89,14,2,59,12700.50,7,0.12,0.69,0.85,3,3.87,60.0,49,3 days 03:05:06.866666666,0.82


In [ ]:
metrics_df['avg_response_time'] = pd.to_timedelta(metrics_df['avg_response_time'])
metrics_df['avg_response_time_hours'] = metrics_df['avg_response_time'].dt.total_seconds() / 3600
metrics_df['avg_response_time_hours']
metrics_df.drop('avg_response_time',axis=1,inplace=True)

In [ ]:
merged_df

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_sell_date,first_sell_month,sell_month,delay,is_delayed,product_category_name
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29,2017-09-13,2017-03,2017-09,-9.0,False,cool_stuff
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15,2017-04-26,2017-04,2017-04,-3.0,False,pet_shop
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,2018-01-14,2017-10,2018-01,-14.0,False,furniture_decor
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20,2018-08-08,2017-07,2018-08,-6.0,False,perfumery
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17,2017-02-04,2016-10,2017-02,-16.0,False,garden_tools
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112096,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,b51593916b4b8e0d6f66f2ae24f2673d,delivered,2018-04-23 13:57:06,2018-04-25 04:11:01,2018-04-25 12:09:00,2018-05-10 22:56:40,2018-05-18,2018-04-23,2017-08,2018-04,-8.0,False,housewares
112097,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53,84c5d4fbaf120aae381fad077416eaa0,delivered,2018-07-14 10:26:46,2018-07-17 04:31:48,2018-07-17 08:05:00,2018-07-23 20:31:55,2018-08-01,2018-07-14,2018-04,2018-07,-9.0,False,computers_accessories
112098,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95,29309aa813182aaddc9b259e31b870e6,delivered,2017-10-23 17:07:56,2017-10-24 17:14:25,2017-10-26 15:13:14,2017-10-28 12:22:22,2017-11-10,2017-10-23,2017-01,2017-10,-13.0,False,sports_leisure
112099,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72,b5e6afd5a41800fdf401e0272ca74655,delivered,2017-08-14 23:02:59,2017-08-15 00:04:32,2017-08-15 19:02:53,2017-08-16 21:59:40,2017-08-25,2017-08-14,2017-05,2017-08,-9.0,False,computers_accessories


### 셀러가 보유한 카테고리 종류

In [ ]:
# NaN 값을 포함한 행 제거
merged_df_cleaned = merged_df.dropna(subset=['product_category_name'])

df_seller_categories = merged_df_cleaned.groupby('seller_id')['product_category_name'].apply(
    lambda x: "{" + ",".join(sorted(set(x))) + "}"
).reset_index()

df_seller_categories

,seller_id,product_category_name
0,0015a82c2db000af6aaaf3ae2ecb0532,{small_appliances}
1,001cca7ae9ae17fb1caed9dfb1094831,"{construction_tools_construction,garden_tools}"
2,002100f778ceb8431b7a1020ff7ab48f,{furniture_decor}
3,004c9cd9d87a3c30c522c48c4fc07416,{bed_bath_table}
4,00720abe85ba0859807595bbf045a33b,{housewares}
...,...,...
2985,ffc470761de7d0232558ba5e786e57b7,"{auto,construction_tools_construction,furnitur..."
2986,ffdd9f82b9a447f6f8d4b91554cc7dd3,"{furniture_decor,housewares}"
2987,ffeee66ac5d5a62fe688b9d26f83f534,{home_appliances}
2988,fffd5413c0700ac820c7069d66d98c89,"{furniture_decor,furniture_living_room,housewa..."


In [ ]:
# NaN 값을 가진 행 제거 (product_category_name, price, freight_value 열)
merged_df_cleaned = merged_df.dropna(subset=['product_category_name', 'price', 'freight_value']).copy()

# 매출 계산 (price + freight_value)
merged_df_cleaned.loc[:, 'sales_revenue'] = merged_df_cleaned['price'] + merged_df_cleaned['freight_value']

# 각 셀러가 보유한 카테고리별 매출 계산
seller_category_sales = merged_df_cleaned.groupby(['seller_id', 'product_category_name'])['sales_revenue'].sum().reset_index(name='total_sales_revenue')

# 각 셀러가 보유한 카테고리의 수 계산
seller_category_count = seller_category_sales.groupby('seller_id')['product_category_name'].count().reset_index(name='category_count')

# 보유한 카테고리가 3개 이상인 셀러는 상위 2개 카테고리 선택
top_2_categories = seller_category_sales.groupby('seller_id').apply(
    lambda x: x.nlargest(2, 'total_sales_revenue') if len(x) > 2 else x).reset_index(drop=True)

df_combined = top_2_categories.groupby('seller_id')['product_category_name'].apply(
    lambda x: "{" + ",".join(sorted(x)) + "}").reset_index(name='product_category')

remaining_sellers = seller_category_sales[~seller_category_sales['seller_id'].isin(df_combined['seller_id'])]
df_remaining = remaining_sellers.groupby('seller_id')['product_category_name'].apply(
    lambda x: "{" + ",".join(sorted(x)) + "}").reset_index(name='product_category')

category_df = pd.concat([df_combined, df_remaining], ignore_index=True)

category_df

,seller_id,product_category
0,0015a82c2db000af6aaaf3ae2ecb0532,{small_appliances}
1,001cca7ae9ae17fb1caed9dfb1094831,"{construction_tools_construction,garden_tools}"
2,002100f778ceb8431b7a1020ff7ab48f,{furniture_decor}
3,004c9cd9d87a3c30c522c48c4fc07416,{bed_bath_table}
4,00720abe85ba0859807595bbf045a33b,{housewares}
...,...,...
2985,ffc470761de7d0232558ba5e786e57b7,"{garden_tools,housewares}"
2986,ffdd9f82b9a447f6f8d4b91554cc7dd3,"{furniture_decor,housewares}"
2987,ffeee66ac5d5a62fe688b9d26f83f534,{home_appliances}
2988,fffd5413c0700ac820c7069d66d98c89,"{furniture_decor,housewares}"


## Public + Funnel

### A 그룹

In [ ]:
# 'seller_id'를 기준으로 두 데이터셋 병합
funnel_sellers = pd.merge(cd, metrics_df, on='seller_id', how='inner')

a_group_sellers = funnel_sellers[funnel_sellers['monetary'].notna()]

a_group_sellers_filtered = a_group_sellers[['seller_id']]

print(f'A그룹 셀러 수: {len(a_group_sellers_filtered)}명')
a_group_sellers_filtered.head()

A그룹 셀러 수: 379명


,seller_id
0,2c43fb513632d29b3b58df74816f1b06
1,612170e34b97004b3ba37eae81836b4c
2,ed8cb7b190ceb6067227478e48cf8dde
3,1c742ac33582852aaf3bcfbf5893abcf
4,44ed138eca6214d572ce1d813fb0049b


### B 그룹

In [ ]:
# 'seller_id'를 기준으로 두 데이터셋 병합
funnel_sellers = pd.merge(cd, metrics_df, on='seller_id', how='inner')

a_group_sellers = funnel_sellers[funnel_sellers['monetary'].notna()]

a_group_sellers_filtered = a_group_sellers[['seller_id']]

remaining_sellers = merged_df[~merged_df['seller_id'].isin(a_group_sellers_filtered['seller_id'])]

remaining_sellers.loc[:, 'first_sell_month'] = pd.to_datetime(remaining_sellers['first_sell_month'], format='%Y-%m', errors='coerce')

b_group_sellers = remaining_sellers[remaining_sellers['first_sell_month'] >= pd.Timestamp('2018-01-01')].drop_duplicates(subset=['seller_id'])

print(f'B그룹 셀러 수: {len(b_group_sellers)}명')

b_group_sellers = b_group_sellers[['seller_id']]

b_group_sellers.head()

B그룹 셀러 수: 884명


,seller_id
7,5996cddab893a4652a15592fb58ab8db
58,5f3ae9136c875522250f8184f253413a
59,e8b4225284fbb02d16f200513f1f395d
74,b1a81260566c1bac3114a6d124413f27
75,00720abe85ba0859807595bbf045a33b


### 층 확인

In [ ]:
test = pd.merge(b_group_sellers, df_seller_categories, on='seller_id', how='inner')
test['product_category_name'].value_counts()

,count
product_category_name,
health_beauty,69
auto,57
sports_leisure,50
housewares,49
furniture_decor,30
...,...
"arts_and_craftmanship,auto",1
"food,health_beauty,small_appliances",1
"auto,costruction_tools_tools,housewares,stationery",1


In [ ]:
test2 = pd.merge(b_group_sellers, category_df, on='seller_id', how='inner')
test2['product_category'].value_counts()

,count
product_category,
{health_beauty},69
{auto},57
{sports_leisure},50
{housewares},49
{furniture_decor},30
...,...
"{auto,construction_tools_safety}",1
"{computers,computers_accessories}",1
"{consoles_games,electronics}",1


### PSM 1:2

In [ ]:
# 성향 점수 매칭에 사용할 컬럼 필터링
psm_columns = [col for col in metrics_df.columns if col not in ['months_with_sale', 'recency', 'frequency', 'monetary', 'seller_id', 'unique_category_count']]

a_group_sellers_filtered['label'] = 1
b_group_sellers['label'] = 0

psm_data = pd.concat([a_group_sellers_filtered, b_group_sellers], axis=0)
psm_data = pd.merge(psm_data, metrics_df[['seller_id'] + psm_columns], on='seller_id', how='left')

# 결측치 제거
psm_data = psm_data.dropna()

# 독립 변수와 종속 변수 설정
X = psm_data[psm_columns]
y = psm_data['label']

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 성향 점수 모델 (로지스틱 회귀 사용)
logit_model = sm.Logit(y, sm.add_constant(X_scaled)).fit()

# 성향 점수 예측
psm_data['propensity_score'] = logit_model.predict(sm.add_constant(X_scaled))

# A그룹과 B그룹을 성향 점수 기반으로 분리
a_group = psm_data[psm_data['label'] == 1]
b_group = psm_data[psm_data['label'] == 0]

# 1:2 비율로 매칭
nbrs = NearestNeighbors(n_neighbors=2).fit(b_group[['propensity_score']])

distances, indices = nbrs.kneighbors(a_group[['propensity_score']])

matched_b_group_sellers_list = []

for i, idx in enumerate(indices):
    matched_b_group_sellers_list.append({
        'a_seller_id': a_group.iloc[i]['seller_id'],
        'b_seller_id_1': b_group.iloc[idx[0]]['seller_id'],
        'b_seller_id_2': b_group.iloc[idx[1]]['seller_id']
    })

matched_results_df = pd.DataFrame(matched_b_group_sellers_list)

# 중복 제거 작업 수행
combined_b_sellers = pd.concat([matched_results_df['b_seller_id_1'], matched_results_df['b_seller_id_2']])
unique_b_sellers = combined_b_sellers.drop_duplicates().reset_index(drop=True)

print(f"중복 제거 후 고유한 B그룹 셀러 수: {len(unique_b_sellers)}")
unique_b_sellers.head()

Optimization terminated successfully.
         Current function value: 0.598788
         Iterations 7
중복 제거 후 고유한 B그룹 셀러 수: 460


,0
0,00720abe85ba0859807595bbf045a33b
1,5a413ade68e8f8d93071a7f52a64cb9e
2,5e25c632f9db6cfb8bc100421c2737b6
3,5996cddab893a4652a15592fb58ab8db
4,e546117ed9cafbc40239c0c78635c584


### ATE, 표준오차

변동성 근거

In [ ]:
# 성향 점수 매칭에 사용할 컬럼 필터링
psm_columns = [col for col in metrics_df.columns if col not in ['months_with_sale', 'recency', 'frequency', 'monetary', 'seller_id', 'unique_category_count']]
outcome_columns = ['months_with_sale', 'recency', 'frequency', 'monetary', 'unique_category_count']

# A그룹 셀러에 label=1, B그룹 셀러에 label=0 부여 (.loc[] 사용)
a_group_sellers_filtered = a_group_sellers_filtered.copy()
b_group_sellers = b_group_sellers.copy()

a_group_sellers_filtered.loc[:, 'label'] = 1
b_group_sellers.loc[:, 'label'] = 0

# A그룹과 B그룹 합치기
psm_data = pd.concat([a_group_sellers_filtered, b_group_sellers], axis=0)

# psm_data에 metrics_df 데이터를 병합 (매칭을 위해 seller_id로)
psm_data = pd.merge(psm_data, metrics_df[['seller_id'] + psm_columns + outcome_columns], on='seller_id', how='left')

# 결측치 제거
psm_data = psm_data.dropna()

# 독립 변수와 종속 변수 설정
X = psm_data[psm_columns]
y = psm_data['label']

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 성향 점수 모델 (로지스틱 회귀 사용)
logit_model = sm.Logit(y, sm.add_constant(X_scaled)).fit()

# 성향 점수 예측
psm_data['propensity_score'] = logit_model.predict(sm.add_constant(X_scaled))

# A그룹과 B그룹을 성향 점수 기반으로 분리
a_group = psm_data[psm_data['label'] == 1]
b_group = psm_data[psm_data['label'] == 0]

# NearestNeighbors 사용하여 매칭 수행 함수
def perform_matching(a_group, b_group, n_neighbors):
    nbrs = NearestNeighbors(n_neighbors=n_neighbors).fit(b_group[['propensity_score']])
    distances, indices = nbrs.kneighbors(a_group[['propensity_score']])
    return indices

# 1:2 매칭 수행
indices_1to2 = perform_matching(a_group, b_group, 2)

# 1:1 매칭 수행
indices_1to1 = perform_matching(a_group, b_group, 1)

# 매칭 결과로 ATE 및 SE 계산 함수
def calculate_ate_se(a_group, b_group, indices, outcome_columns, n_neighbors):
    ate_list = {col: [] for col in outcome_columns}

    for i, idx in enumerate(indices):
        a_effects = psm_data.loc[psm_data['seller_id'] == a_group.iloc[i]['seller_id'], outcome_columns].values[0]

        if n_neighbors == 1:
            b_effects = psm_data.loc[psm_data['seller_id'] == b_group.iloc[idx[0]]['seller_id'], outcome_columns].values[0]
            for j, col in enumerate(outcome_columns):
                ate_list[col].append(a_effects[j] - b_effects[j])
        else:
            b_effects_1 = psm_data.loc[psm_data['seller_id'] == b_group.iloc[idx[0]]['seller_id'], outcome_columns].values[0]
            b_effects_2 = psm_data.loc[psm_data['seller_id'] == b_group.iloc[idx[1]]['seller_id'], outcome_columns].values[0]
            b_effects_avg = (b_effects_1 + b_effects_2) / 2
            for j, col in enumerate(outcome_columns):
                ate_list[col].append(a_effects[j] - b_effects_avg[j])

    ate = {col: np.mean(ate_list[col]) for col in outcome_columns}
    se = {col: np.std(ate_list[col]) / np.sqrt(len(ate_list[col])) for col in outcome_columns}

    return ate, se

# 1:1 매칭 ATE 및 SE 계산
ate_1to1, se_1to1 = calculate_ate_se(a_group, b_group, indices_1to1, outcome_columns, 1)

# 1:2 매칭 ATE 및 SE 계산
ate_1to2, se_1to2 = calculate_ate_se(a_group, b_group, indices_1to2, outcome_columns, 2)

# 결과 출력
for col in outcome_columns:
    print(f"1:1 매칭 ATE ({col}): {ate_1to1[col]}, 표준 오차: {se_1to1[col]}")
    print(f"1:2 매칭 ATE ({col}): {ate_1to2[col]}, 표준 오차: {se_1to2[col]}")

Optimization terminated successfully.
         Current function value: 0.598788
         Iterations 7
1:1 매칭 ATE (months_with_sale): -0.21108179419525067, 표준 오차: 0.11594837998044144
1:2 매칭 ATE (months_with_sale): -0.23087071240105542, 표준 오차: 0.09452941814206545
1:1 매칭 ATE (recency): -12.224274406332453, 표준 오차: 3.3009718156958257
1:2 매칭 ATE (recency): -12.220316622691293, 표준 오차: 2.6686927501002353
1:1 매칭 ATE (frequency): -1.5989445910290236, 표준 오차: 2.4317831977959457
1:2 매칭 ATE (frequency): -2.179419525065963, 표준 오차: 2.138635631009767
1:1 매칭 ATE (monetary): -225.6694459102902, 표준 오차: 573.9291106666502
1:2 매칭 ATE (monetary): -498.6285092348285, 표준 오차: 539.8461179093159
1:1 매칭 ATE (unique_category_count): 0.0712401055408971, 표준 오차: 0.11690400880702938
1:2 매칭 ATE (unique_category_count): 0.11081794195250659, 표준 오차: 0.08952898308538335


### SMD

선택편향 근거

In [ ]:
# SMD 계산 함수
def calculate_smd(a_group, b_group, columns):
    smd_list = {}

    for col in columns:
        mean_a = a_group[col].mean()
        mean_b = b_group[col].mean()
        pooled_std = np.sqrt((a_group[col].var() + b_group[col].var()) / 2)

        smd = np.abs(mean_a - mean_b) / pooled_std
        smd_list[col] = smd

    return smd_list

# 매칭 전후 처리군과 대조군의 공변량 균형 평가
def perform_smd_analysis(a_group, b_group, indices_1to1, indices_1to2, psm_columns):
    # 매칭 전 SMD 계산
    smd_before = calculate_smd(a_group, b_group, psm_columns)

    # 1:1 매칭 후 SMD 계산
    b_group_matched_1to1 = b_group.iloc[indices_1to1[:, 0]].reset_index(drop=True)
    smd_1to1_after = calculate_smd(a_group, b_group_matched_1to1, psm_columns)

    # 1:2 매칭 후 SMD 계산
    b_group_matched_1to2_1 = b_group.iloc[indices_1to2[:, 0]].reset_index(drop=True)
    b_group_matched_1to2_2 = b_group.iloc[indices_1to2[:, 1]].reset_index(drop=True)

    # 두 대조군의 평균을 계산할 때 숫자 데이터만 처리하도록 수정
    numeric_columns = b_group_matched_1to2_1.select_dtypes(include=[np.number]).columns
    b_group_matched_1to2_avg = pd.concat([b_group_matched_1to2_1[numeric_columns], b_group_matched_1to2_2[numeric_columns]]).groupby(level=0).mean()

    smd_1to2_after = calculate_smd(a_group, b_group_matched_1to2_avg, numeric_columns)

    return smd_before, smd_1to1_after, smd_1to2_after

# SMD 분석 수행
smd_before, smd_1to1_after, smd_1to2_after = perform_smd_analysis(a_group, b_group, indices_1to1, indices_1to2, psm_columns)

def print_smd_results(title, smd_dict):
    print(f"\n{title}")
    print("-" * 50)
    for variable, smd in sorted(smd_dict.items()):
        print(f"{variable:30}: {smd:.6f}")

print_smd_results("매칭 전 SMD:", smd_before)
print_smd_results("1:1 매칭 후 SMD:", smd_1to1_after)
print_smd_results("1:2 매칭 후 SMD:", smd_1to2_after)


매칭 전 SMD:
--------------------------------------------------
avg_response_time_hours       : 0.106320
avg_review_score              : 0.175307
delay_rate                    : 0.100234
delayed_count                 : 0.017185
payment_concentration         : 0.125680
payment_mix_index             : 0.180130
total_reviews_x               : 0.055007
within_2_days_count           : 0.052244
within_2_days_rate            : 0.056485

1:1 매칭 후 SMD:
--------------------------------------------------
avg_response_time_hours       : 0.022663
avg_review_score              : 0.055740
delay_rate                    : 0.069819
delayed_count                 : 0.059274
payment_concentration         : 0.059731
payment_mix_index             : 0.048320
total_reviews_x               : 0.049316
within_2_days_count           : 0.051043
within_2_days_rate            : 0.049710

1:2 매칭 후 SMD:
--------------------------------------------------
avg_response_time_hours       : 0.005440
avg_review_score           

<ipython-input-12-2fde5b079e10>:10: RuntimeWarning: divide by zero encountered in scalar divide
  smd = np.abs(mean_a - mean_b) / pooled_std


### 층화 표본 추출

In [ ]:
categories_df['product_category'] = categories_df['product_category'].str.strip('{}').str.replace(' ', '')

categories_df['product_category'] = categories_df['product_category'].apply(lambda x: ','.join(sorted(x.split(','))) if ',' in x else x)

a_group_sellers_full = pd.merge(a_group_sellers_filtered, categories_df, on='seller_id', how='left')

a_group_sellers_full = a_group_sellers_full.dropna(subset=['product_category'])

a_group_category_distribution = a_group_sellers_full['product_category'].value_counts(normalize=True)

unique_b_sellers_df = pd.DataFrame(unique_b_sellers, columns=['seller_id'])

b_group_sellers_full = pd.merge(unique_b_sellers_df, categories_df, on='seller_id', how='left')

b_group_sellers_full = b_group_sellers_full.dropna(subset=['product_category'])

b_group_sampled = pd.DataFrame()

for category, ratio in a_group_category_distribution.items():
    category_group = b_group_sellers_full[b_group_sellers_full['product_category'] == category]

    n_samples = min(len(category_group), int(len(a_group_sellers_filtered) * ratio))

    if n_samples > 0:
        sampled = category_group.sample(n=n_samples, replace=False)
        b_group_sampled = pd.concat([b_group_sampled, sampled])

print(f"층화 표본 추출된 B그룹 셀러 수: {len(b_group_sampled)}")
b_group_sampled

층화 표본 추출된 B그룹 셀러 수: 250


,seller_id,product_category
147,461ff5303b6c9c1a4a07f40c5aece516,health_beauty
39,e0b162a693bc7f9eab00f783a31d60ea,health_beauty
98,87b4428a334b8f676fd9a7f3bf544fb5,health_beauty
288,be7a093e5b4d147b36b72bb1f9e415e5,health_beauty
457,258447aad3f7b01e2643c89f64dc5647,health_beauty
...,...,...
379,d6ab728cf30867cf3f1a2e216ae2f9b7,"signaling_and_security,sports_leisure"
196,392353362d22cc2c236e1ee81ff19890,"construction_tools_construction,home_construction"
207,8fb791022c1fc8909664f48ab7dc636d,home_appliances_2
345,7f7b8245c336066a1f9933c359f11d77,"health_beauty,toys"


In [ ]:
b_group_sampled['product_category'].value_counts()

,count
product_category,
health_beauty,34
housewares,26
auto,21
sports_leisure,13
bed_bath_table,12
...,...
home_construction,1
construction_tools_safety,1
food_drink,1


### 지표 확인

In [ ]:
# ['months_with_sale', 'frequency', 'monetary', 'seller_id', 'unique_category_count']

a_group_metrics = pd.merge(a_group_sellers_filtered, metrics_df, on='seller_id', how='left')
b_group_metrics = pd.merge(b_group_sampled[['seller_id']], metrics_df, on='seller_id', how='left')

display(a_group_metrics.drop_duplicates(subset=['seller_id']).mean(numeric_only=True).round(3)) # 실험군
display(b_group_metrics.drop_duplicates(subset=['seller_id']).mean(numeric_only=True).round(3)) # 통제군

,0
label,1.000
months_with_sale,2.715
recency,32.055
frequency,11.900
monetary,2559.265
delayed_count,0.644
delay_rate,0.046
payment_mix_index,0.577
payment_concentration,0.823
unique_category_count,1.852


,0
months_with_sale,2.704
recency,46.900
frequency,10.900
monetary,2533.793
delayed_count,0.608
delay_rate,0.055
payment_mix_index,0.502
payment_concentration,0.846
unique_category_count,1.376
avg_review_score,4.296


### 검증

In [ ]:
a_group_metrics

,seller_id,label,months_with_sale,recency,frequency,monetary,delayed_count,delay_rate,payment_mix_index,payment_concentration,unique_category_count,avg_review_score,total_reviews_x,within_2_days_count,within_2_days_rate,avg_response_time_hours
0,2c43fb513632d29b3b58df74816f1b06,1,2,36,3,930.17,0,0.00,-0.00,1.00,2,3.67,3.0,2,0.67,44.238241
1,612170e34b97004b3ba37eae81836b4c,1,3,7,107,26144.20,1,0.01,1.05,0.79,12,4.43,110.0,77,0.70,40.184811
2,ed8cb7b190ceb6067227478e48cf8dde,1,2,2,15,1060.66,1,0.07,1.40,0.53,3,3.18,17.0,16,0.94,27.580163
3,1c742ac33582852aaf3bcfbf5893abcf,1,1,67,1,113.69,0,0.00,-0.00,1.00,1,5.00,1.0,1,1.00,20.251111
4,44ed138eca6214d572ce1d813fb0049b,1,1,8,2,298.54,0,0.00,1.00,0.50,2,4.67,3.0,2,0.67,52.876204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,f46490624488d3ff7ce78613913a7711,1,4,7,83,9189.43,9,0.11,1.27,0.67,1,3.46,91.0,54,0.59,51.020192
375,7c9130f5f3729edd32aa059141d92484,1,2,6,3,294.97,0,0.00,-0.00,1.00,1,3.75,4.0,1,0.25,65.443611
376,33dd941c27854f7625b968cc6195a552,1,3,26,5,7168.41,0,0.00,0.72,0.80,1,4.40,5.0,3,0.60,48.137944
377,c70a353f02429c00775a46a75fb787da,1,2,18,6,336.94,0,0.00,1.75,0.50,1,4.67,6.0,6,1.00,20.029167


In [ ]:
b_group_metrics

,seller_id,months_with_sale,recency,frequency,monetary,delayed_count,delay_rate,payment_mix_index,payment_concentration,unique_category_count,avg_review_score,total_reviews_x,within_2_days_count,within_2_days_rate,avg_response_time_hours
0,6e386f64c84e482f0b9abf960797ee1b,6,12,30,13563.66,2,0.07,0.98,0.73,1,4.61,28.0,19,0.68,52.682083
1,fad44952713764836814be105382aee5,7,15,32,5743.71,0,0.00,1.34,0.62,1,4.28,32.0,24,0.75,34.076510
2,10264f60a8f0a4d2f30810f5c661c673,3,20,4,326.46,0,0.00,0.81,0.75,1,5.00,4.0,3,0.75,24.703403
3,b6f52e346160519c24a23af05410ad32,1,125,1,143.25,0,0.00,-0.00,1.00,1,4.00,1.0,1,1.00,10.610833
4,3e46a38ee171c503c3b4a3f23fe3bf0c,1,68,1,69.01,0,0.00,-0.00,1.00,1,5.00,1.0,1,1.00,1.886944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,d6ab728cf30867cf3f1a2e216ae2f9b7,6,7,12,2532.32,0,0.00,0.41,0.92,2,4.43,14.0,10,0.71,39.406151
246,392353362d22cc2c236e1ee81ff19890,3,6,5,1234.85,0,0.00,0.97,0.60,2,4.60,10.0,3,0.30,85.279250
247,8fb791022c1fc8909664f48ab7dc636d,1,37,5,8483.35,0,0.00,-0.00,1.00,1,4.20,5.0,5,1.00,38.284667
248,7f7b8245c336066a1f9933c359f11d77,5,12,38,3249.67,1,0.03,1.19,0.71,8,4.56,43.0,28,0.65,45.852713


In [ ]:
def calculate_smd(group1, group2):
    mean_diff = np.mean(group1) - np.mean(group2)
    pooled_std = np.sqrt((np.std(group1, ddof=1) ** 2 + np.std(group2, ddof=1) ** 2) / 2)
    smd = mean_diff / pooled_std
    return smd

# 빈 리스트 생성
smd_results = []

# 변수별 SMD 계산 및 결과 저장
for variable in psm_columns:
    smd_value = calculate_smd(b_group_metrics[variable], a_group_metrics[variable])
    smd_results.append({
        "Variable": variable,
        "Control Group Mean (SD)": f"{np.mean(b_group_metrics[variable]):.2f} ({np.std(b_group_metrics[variable], ddof=1):.2f})",
        "Experimental Group Mean (SD)": f"{np.mean(a_group_metrics[variable]):.2f} ({np.std(a_group_metrics[variable], ddof=1):.2f})",
        "SMD": smd_value.round(5)
    })

# DataFrame 생성 및 출력
smd_df = pd.DataFrame(smd_results)
smd_df.set_index("Variable", inplace=True)
smd_df

,Control Group Mean (SD),Experimental Group Mean (SD),SMD
Variable,,,
delayed_count,0.61 (1.85),0.64 (3.51),-0.01277
delay_rate,0.05 (0.15),0.05 (0.14),0.05728
payment_mix_index,0.50 (0.51),0.58 (0.51),-0.14749
payment_concentration,0.85 (0.17),0.82 (0.17),0.13231
avg_review_score,4.30 (0.75),4.27 (0.76),0.02781
total_reviews_x,12.21 (33.68),13.25 (37.08),-0.02917
within_2_days_count,8.88 (24.22),9.52 (25.81),-0.02545
within_2_days_rate,0.72 (0.29),0.73 (0.27),-0.00524
avg_response_time_hours,43.06 (36.65),43.16 (31.71),-0.00304


In [ ]:
inter_variables = ['recency', 'frequency', 'monetary','months_with_sale','unique_category_count']
# for variable in inter_variables:
#   t_stat, p_value = stats.ttest_ind(b_group_metrics[variable], a_group_metrics[variable])
#   print(f"T-statistic for {variable}: {t_stat.round(5)}")
#   print(f"P-value for {variable}: {p_value.round(5)}")
# # t 통계량이 클수록 두 그룹 간 차이가 큰 것
# # p-value 0.05이하이면 통계적으로 유의함
# # 통계적으로 유의미한 것: recency, unique_category_count
# # 개선된 컨설팅 이후 최근까지 구매하는 경향을 보이기는 하지만, frequency와 monetary은 큰 차이가 없다.
# # 대대적인 개선이 필요

T-statistic for recency: 4.03656
P-value for recency: 6e-05
T-statistic for frequency: -0.37176
P-value for frequency: 0.7102
T-statistic for monetary: -0.03932
P-value for monetary: 0.96865
T-statistic for months_with_sale: -0.07781
P-value for months_with_sale: 0.93801
T-statistic for unique_category_count: -4.3115
P-value for unique_category_count: 2e-05


In [ ]:
results = []
from scipy.stats import mannwhitneyu
for variable in inter_variables:
    stat_cd = b_group_metrics[variable].mean(), b_group_metrics[variable].std()
    stat_ex = a_group_metrics[variable].mean(), a_group_metrics[variable].std()
    stat, p_value = mannwhitneyu(b_group_metrics[variable], a_group_metrics[variable], alternative='two-sided')
    significance = "True" if p_value < 0.05 else "False"
    results.append({
        'Variable': variable,
        'Control Group Mean (SD)': f"{stat_cd[0]:.2f} ({stat_cd[1]:.2f})",
        'Experimental Group Mean (SD)': f"{stat_ex[0]:.2f} ({stat_ex[1]:.2f})",
        'U-statistic': round(stat, 5),
        'P-value': round(p_value, 5),
        'Significance': significance
    })
results_df = pd.DataFrame(results)
results_df

,Variable,Control Group Mean (SD),Experimental Group Mean (SD),U-statistic,P-value,Significance
0,recency,46.90 (54.62),32.06 (37.60),54403.5,0.00162,True
1,frequency,10.90 (30.41),11.90 (34.61),43396.5,0.07194,False
2,monetary,2533.79 (7305.86),2559.26 (8347.81),45290.0,0.34999,False
3,months_with_sale,2.70 (1.95),2.72 (1.59),44551.5,0.19347,False
4,unique_category_count,1.38 (0.97),1.85 (1.56),38038.0,0.00000,True
